In [3]:
!pip install requests
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 1.7 MB/s eta 0:00:00


# React Agent from scratch

In [34]:
import datetime
import requests
from zoneinfo import ZoneInfo
from abc import ABC, abstractmethod
import json

class Tool(ABC):
    @abstractmethod
    def name(self) -> str:
        pass

    @abstractmethod
    def description(self) -> str:
        pass

    @abstractmethod
    def use(self, *args, **kwargs):
        pass

class TimeTool(Tool):
    def name(self):
        return "Time Tool"

    def description(self):
        return "Provides the current time for a given city's timezone like Asia/Kolkata, America/New_York etc. If no timezone is provided, it returns the local time."

    def args(self):
        return "{'timezone': ''}"

    def use(self, args, **kwargs):
        format = "%Y-%m-%d %H:%M:%S %Z%z"
        current_time = datetime.datetime.now()
        try:
          input_timezone = args['timezone']
          if input_timezone:
              print("TimeZone", input_timezone)
              current_time =  current_time.astimezone(ZoneInfo(input_timezone))
        except Exception as e:
          pass
        return f"Couldn't fetch timezone details."

class FlightDetails(Tool):
    def name(self):
        return "Flight details Tool"

    def description(self):
        return "Provides flight details between the provided cities(origin and destination) and given date."

    def args(self):
        return "{'origin':'','destination':'','date':''}"

    def use(self, args, **kwargs):
        try:
          src = args['origin']
          dest = args['destination']
          date = args['date']
          if not src or not dest:
            return "Provide details of your origin and destination airports"
          if not date:
            return "Provide dates for your trip."
          else:
            return f"""{src} - {dest} - {date} Flight details: \n
                    1) Indigo : 7:00 AM
                    2) AirIndia: 8:00 AM """
        except Exception as e:
          return "Couldn't fetch flight details"

class WeatherTool(Tool):
    def name(self):
        return "Weather Tool"

    def description(self):
        return "Provides weather information for a given location"

    def args(self):
        return "{'location': ''}"

    def use(self, args, **kwargs):
      try:
        location = args['location']
        temp = 35
        return f"The weather in {location} is currently {temp}°C."
      except:
        return f"Couldn't fetch weather details for {location}"

class AskUser(Tool):
    def name(self):
        return "Input Tool"

    def description(self):
        return "Get inputs from a user"

    def args(self):
        return "{'question': ''}"

    def use(self, args, **kwargs):
      try:
        question = args['question']
        answer = input(f"Question: {question}\n Your Answer: ")
        return f"The answer to this question {question} is:  {answer}"
      except:
        return f"Couldn't get the answer"

class PopulationLookup(Tool):
    def name(self):
        return "Population Tool"

    def description(self):
        return "Get population for the given country"

    def args(self,country=""):
        json_dict = {'country':country}
        json_str = json.dumps(json_dict)
        return json_str

    def use(self, args, **kwargs):
      try:
        country = args['country']
        if country=="India":
          return f"Population of this country is 5500000"
        if country=="USA":
          return f"Population of this country is 300000"
        if country=="UK":
          return f"Population of this country is 1500000"

      except:
        return f"Couldn't get the answer"

class MathTool(Tool):
    def name(self):
        return "Math Tool"

    def description(self):
        return "Math Tool"

    def args(self,operation="",inputs="[list of integers]"):
        json_dict = {'operation': operation,'inputs':inputs}
        json_str = json.dumps(json_dict)
        return json_str

    def use(self, args, **kwargs):
      try:
        if args["operation"]=="+":
          inputs = args["inputs"]
          return int(inputs[0]) + int(inputs[1])
        else:
          return 150

      except:
        return f"Couldn't get the answer"


In [32]:
PopulationLookup().args("India")

'{"country": "India"}'

In [35]:
import ast
import re
import os
import json
import requests
from pydantic import BaseModel, Field
from typing import Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

class Agent:
    def __init__(self):
        self.tools = []
        self.memory = []
        self.messages = []
        self.max_memory = 10
        self.llm = ChatOpenAI(model="gpt-4o-mini")

    def add_tool(self, tool: Tool):
        self.tools.append(tool)

    def json_parser(self, input_string):
      try:
        json_dict = eval(input_string)
      except:
        return "Invalid JSON response"

      if isinstance(json_dict, dict):
        return json_dict

      raise "Invalid JSON response"

    def parse_response(self,response: str):
        """
        Parse the LLM response to extract and validate JSON content.
        """
        stack = []
        result = ""
        for i, char in enumerate(response):
            if char == "{":
                if not stack:  # Start of the outermost block
                    start = i
                stack.append("{")
            elif char == "}":
                stack.pop()
                if not stack:  # End of the outermost block
                    result = response[start:i+1]
                    break

        response_dict = self.json_parser(result)

        return response_dict

    def get_system_prompt(self):

        tool_descriptions = "\n".join([f"- {tool.name()}: {tool.description()}" for tool in self.tools])
        tool_args = "\n".join([f"- {tool.name()}: {tool.args()}" for tool in self.tools])
        tool_names = ",".join([tool.name() for tool in self.tools])
        system_prompt = f'''Answer the following questions as best you can. You have access to the following tools:

                      {tool_descriptions}

                      Use the following format:

                      Thought: you should always think about what to do
                      Action: the action to take, should be one of the tools : [{tool_names}]
                      Action Input: the input to the action. Argument format is [{tool_args}]
                      PAUSE
                      You will be called again with this:
                      Observation: the result of the action
                      ... (this Thought/Action/Action Input/Observation can repeat N times)
                      Thought: I now know the final answer
                      Final Answer: the final answer to the original input question

                      If you decide to answer directly without using any tools then reply in "Final Answer" as your response.
                      '''
        return system_prompt

    def process_input(self):

        if len(self.messages)>self.max_memory:
          system_msg = self.messages[0]
          self.messages = [system_msg] + self.messages[-self.max_memory:]

        response = self.llm.invoke(self.messages)

        if "Action" in response.content and "Action Input" in response.content:
          temp_chunk = response.content.split("Action:")
          thought = temp_chunk[0].split("Thought:")[-1].strip()
          print("Thought:",thought)
          action_name = temp_chunk[-1].split("Action Input:")[0].strip()
          print("Action name: ",action_name)
          action_args = self.parse_response(response.content)
          print("Action input: ",action_args)
          for tool in self.tools:
              if tool.name().lower() == action_name.lower():
                  tool_res = tool.use(action_args)
                  print("Observation: ",tool_res)
                  return "continue",response.content + f"\n Observation: {tool_res}"
          print("--------------------------")

        if "Final Answer:" in response.content:
          return "stop",response.content.split("Final Answer:")[-1].strip()

        return "stop",response.content

    def run(self):
      print("LLM Agent: Hello! How can I assist you today?")
      user_input = input("User: ")

      self.messages = [
                  ("system", self.get_system_prompt()),
                  ("human", user_input ),
              ]
      i = 1

      while True:
        i+=1
        if i==20:
          break

        if user_input and user_input.lower() in ["exit", "bye", "close"]:
          print("See you later!")
          break

        flag, response = self.process_input()
        if flag == "continue":
            self.messages.append(("assistant",response))
        else:
            print("LLM Agent: ", response)
            self.messages.append(("assistant",response))
            user_input = input("\nUser: ")
            self.messages.append(("human",user_input))

In [20]:
from google.colab import userdata

def main():
    agent = Agent()
    agent.add_tool(TimeTool())
    agent.add_tool(WeatherTool())
    agent.add_tool(FlightDetails())
    agent.add_tool(AskUser())
    agent.add_tool(MathTool())
    agent.add_tool(PopulationLookup())
    agent.run()

if __name__ == "__main__":
    main()

LLM Agent: Hello! How can I assist you today?
User: whats the weather today?
Thought: I need to know the location for which the weather is being requested.
Action name:  Input Tool
Action input:  {'question': 'Please provide the location for which you want to know the weather.'}
Question: Please provide the location for which you want to know the weather.
 Your Answer: Mumbai
Observation:  The answer to this question Please provide the location for which you want to know the weather. is:  Mumbai
Thought: Now that I have the location, I can check the weather for Mumbai.
Action name:  Weather Tool
Action input:  {'location': 'Mumbai'}
Observation:  The weather in Mumbai is currently 35°C.
LLM Agent:  The weather today in Mumbai is currently 35°C.

User: bye
See you later!


# React Agent using Langchain

In [ ]:
%pip install -U langchain_openai langchain-community langgraph langchain-anthropic

In [ ]:
import getpass
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="pr-smug-toenail-81"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user's request only based on the given context."),
    ("user", "Question: {question}\nContext: {context}")
])
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

question = "Can you summarize this morning's meetings?"
context = "During this morning's meeting, we solved all world conflict."
chain.invoke({"question": question, "context": context})

"This morning's meeting successfully addressed and resolved all world conflicts."

In [ ]:
import openai
from langsmith import wrappers, traceable

# Auto-trace LLM calls in-context
client = wrappers.wrap_openai(openai.Client())

@traceable # Auto-trace this function
def pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-4o-mini"
    )
    return result.choices[0].message.content

pipeline("Hello, world!")
# Out:  Hello there! How can I assist you today?

'Hello! How can I assist you today?'

In [ ]:
from langchain.agents import tool


@tool
def search(location: str) -> int:
    """Returns the weather of the given location."""
    return "35 C"

tools = [search]

In [ ]:
import getpass
import os

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
print(response.content)

Hello! How can I assist you today?


In [ ]:
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [ ]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'search', 'args': {'location': 'San Francisco'}, 'id': 'call_HUcavdRvpG2jMU5ByrAul1QA', 'type': 'tool_call'}]


In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [ ]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
for m in response["messages"]:
  print(m)

content='whats the weather in sf?' additional_kwargs={} response_metadata={} id='06955993-6cdd-40e7-ab2a-e64506d35e37'
content='' additional_kwargs={'tool_calls': [{'id': 'call_ViO9kaJUql9evnDllSoQZcTf', 'function': {'arguments': '{"location":"San Francisco"}', 'name': 'search'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 51, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f2cd28694a', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-e8ca1f16-b82a-4f21-bb08-2090dd62f63a-0' tool_calls=[{'name': 'search', 'args': {'location': 'San Francisco'}, 'id': 'call_ViO9kaJUql9evnDllSoQZcTf', 'type': 'tool_call'}] usage_metadata={'input_tokens': 51, 'output_tokens': 15, 'total_to